In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
import gensim.downloader as api
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [3]:
word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведим слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [4]:
# выведем лишнее слово
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))
print(word_vectors.doesnt_match("black green summer brown".split()))

cereal
summer


In [5]:
# определим схожесть между словами
similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('human', 'man')
print(similarity)

similarity = word_vectors.similarity('bee', 'man')
print(similarity)

0.8323494
0.52885115
0.21199903


In [6]:
# найдем top-3 самых близких слов
result = word_vectors.similar_by_word("man", topn=3)
print(result)

result = word_vectors.similar_by_word("cat", topn=3)
print(result)

result = word_vectors.similar_by_word("whore", topn=3)
print(result)

[('woman', 0.832349419593811), ('boy', 0.7914870977401733), ('one', 0.7788748145103455)]
[('dog', 0.8798074126243591), ('rabbit', 0.7424427270889282), ('cats', 0.732300341129303)]
[('slut', 0.6932610273361206), ('bitch', 0.6757593750953674), ('hypocrite', 0.5972879528999329)]


In [7]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [10]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w", encoding='utf-8') as fout:
    with open("Otvety.txt", "r", encoding='utf-8') as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [11]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [15]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r", encoding='utf-8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

In [16]:
sentences = [i for i in sentences if len(i) > 2]

In [17]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

******************

In [18]:
Word2Vec?

In [20]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1,  workers=8)
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [23]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r", encoding='utf-8') as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

True

In [24]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [34]:
TEXT = 'черный кот'
get_response(TEXT, w2v_index, modelW2V.wv, index_map)

['Черная кошка - эта примета еще из язычества: люди верили, что ведьмы превращаются в черных кошек, а если тебе такая "ведьма" дорогу перебегает - это она тебе хочет несчастья. У меня самого жили 2 черные кошки - ничего, жив еще))). \n',
 'Налей в рюмку чай и увидишь!. \n',
 'Кабачки можно и нужно замораживать, а также тыкву (очень полезная каша с ней) , перец очищенный (чтобы зимой фаршировать можно было) , люди замораживают морковь тертую, и даже резаные свежие огурцы. Вот так!. \n']